In [1]:
import asyncio

from joblib import Parallel, delayed

import requests as req
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import time
import multiprocessing as mp

import pandas as pd
pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

from tqdm.notebook import tqdm

#from webdriver_manager.chrome import ChromeDriverManager
#PATH=ChromeDriverManager().install()

PATH=r'Chromedriver\chromedriver'

import warnings
warnings.filterwarnings('ignore')

# stats = stats_first_clean = stats_chess.com

In [2]:
stats_original = pd.read_csv(r'stats_first_clean.csv')
stats = stats_original.copy()
stats.head()

,Unnamed: 0,name,username,profile,user_id,title,fide,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,0,"Engine, Komodo",komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0


In [3]:
stats.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1508 non-null   int64  
 1   name                   1241 non-null   object 
 2   username               1508 non-null   object 
 3   profile                1508 non-null   object 
 4   user_id                1508 non-null   int64  
 5   title                  1508 non-null   object 
 6   fide                   913 non-null    float64
 7   country                1508 non-null   object 
 8   current_rapid_rating   1055 non-null   float64
 9   highest_rapid_rating   997 non-null    float64
 10  rapid_win              1055 non-null   float64
 11  rapid_draw             1055 non-null   float64
 12  rapid_loss             1055 non-null   float64
 13  current_blitz_rating   1388 non-null   float64
 14  highest_blitz_rating   1382 non-null   float64
 15  blit

In [4]:
stats.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1508.0,7.535000e+02,4.354664e+02,0.0,376.75,753.5,1130.25,1507.0
user_id,1508.0,4.145837e+07,4.256272e+07,3453.0,15665282.00,29307905.0,55585392.00,299430199.0
fide,913.0,2.002938e+03,1.061741e+03,0.0,2405.00,2523.0,2600.00,3411.0
current_rapid_rating,1055.0,2.382518e+03,3.165229e+02,263.0,2306.00,2455.0,2567.50,2948.0
highest_rapid_rating,997.0,2.555735e+03,2.535907e+02,1200.0,2470.00,2590.0,2715.00,3045.0
rapid_win,1055.0,3.567299e+01,8.903462e+01,0.0,4.00,12.0,31.00,1561.0
rapid_draw,1055.0,1.287014e+01,4.168809e+01,0.0,1.00,4.0,12.00,1150.0
rapid_loss,1055.0,1.578389e+01,4.330288e+01,0.0,2.00,6.0,16.00,878.0
current_blitz_rating,1388.0,2.696158e+03,2.428279e+02,255.0,2565.00,2725.0,2860.25,3287.0
highest_blitz_rating,1382.0,2.797418e+03,2.223825e+02,1265.0,2680.25,2824.5,2948.00,3332.0


In [5]:
# drop profile and user_id
stats = stats.drop(['profile', 'user_id', 'username', 'title', 'fide'], axis=1)
stats

,Unnamed: 0,name,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,0,"Engine, Komodo",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,1503,"Zvonko, Stanojoski",North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0,2102.0,2083.0,13.0,1.0,7.0
1504,1504,"Gurevich, Dmitry",United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0,2400.0,2594.0,496.0,47.0,402.0
1505,1505,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0,NaN,NaN,NaN,NaN,NaN
1506,1506,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0,NaN,NaN,NaN,NaN,NaN


In [6]:
stats.columns = stats.columns.str.capitalize()
stats

,Unnamed: 0,Name,Country,Current_rapid_rating,Highest_rapid_rating,Rapid_win,Rapid_draw,Rapid_loss,Current_blitz_rating,Highest_blitz_rating,Blitz_win,Blitz_draw,Blitz_loss,Current_bullet_rating,Highest_bullet_rating,Bullet_win,Bullet_draw,Bullet_loss
0,0,"Engine, Komodo",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,1503,"Zvonko, Stanojoski",North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0,2102.0,2083.0,13.0,1.0,7.0
1504,1504,"Gurevich, Dmitry",United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0,2400.0,2594.0,496.0,47.0,402.0
1505,1505,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0,NaN,NaN,NaN,NaN,NaN
1506,1506,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0,NaN,NaN,NaN,NaN,NaN


In [7]:
# drop bullet columns, it's irrevelant for our study
stats = stats.drop(['Current_bullet_rating', 'Highest_bullet_rating', 'Bullet_win', 'Bullet_draw', 'Bullet_loss', 'Unnamed: 0'], axis=1)
stats

,Name,Country,Current_rapid_rating,Highest_rapid_rating,Rapid_win,Rapid_draw,Rapid_loss,Current_blitz_rating,Highest_blitz_rating,Blitz_win,Blitz_draw,Blitz_loss
0,"Engine, Komodo",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Plát, Vojtěch",Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0
2,"Carlsen, PlayMagnus",Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Carlsen, Magnus",Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0
4,"Caruana, Fabiano",United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1503,"Zvonko, Stanojoski",North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0
1504,"Gurevich, Dmitry",United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0
1505,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0
1506,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0


In [8]:
stats = stats.to_csv(r'stats_chess_com.csv', index=False)

# df = top_100_scraping = draws/wins/losses (live torunaments)'yotachess'

In [9]:
df_original = pd.read_csv(r'top_100_scraping.csv')
df = df_original.copy()
df.head()

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Ding, Liren",760 (33.48 %),413 (47.4 %),434 (19.12 %)
4,"Firouzja, Alireza",1484 (52.68 %),223 (23.82 %),662 (23.5 %)


In [10]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    100 non-null    object
 1   Wins    71 non-null     object
 2   Draws   71 non-null     object
 3   Losses  71 non-null     object
dtypes: object(4)
memory usage: 24.6 KB


# table = rating_top_100 = actual rating in all formats (oficial elo)'2700Chess'

In [11]:
table_original = pd.read_csv(r'rating_top_100.csv')
table = table_original.copy()
table.head()

,#,Name,Classic,Rapid,Blitz,Age
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35
3,4,"Ding, Liren",2780.0,2830.0,2787 i,30
4,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20


# data = scraping_top_100 = draws/wins/losses (better scraping, two more rows than df, 'wtf?')

In [12]:
data_original = pd.read_csv(r'escrapeo_top_100.csv')
data = data_original.copy()
data.head()

,Jugador,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Ding, Liren",NaN,NaN,NaN


In [13]:
# renombro las columnas
data.columns = ['name', 'Wins', 'Draws', 'Losses']
data

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Ding, Liren",NaN,NaN,NaN
...,...,...,...,...
97,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
98,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
99,"Malakhov, Vladimir",919 (41.03 %),None (44.29 %),329 (14.69 %)
100,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [14]:
data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    102 non-null    object
 1   Wins    93 non-null     object
 2   Draws   93 non-null     object
 3   Losses  93 non-null     object
dtypes: object(4)
memory usage: 27.4 KB


# complete data

In [15]:
# rows with 'NaN' values
Wins_filtradas = data[data.isnull().any(axis=1)]
Wins_filtradas

,name,Wins,Draws,Losses
2,"Nakamura, Hikaru",NaN,NaN,NaN
4,"Ding, Liren",NaN,NaN,NaN
5,"Firouzja, Alireza",NaN,NaN,NaN
16,"Mamedyarov, Shakhriyar",NaN,NaN,NaN
18,"Le, Quang Liem",NaN,NaN,NaN
19,"Grischuk, Alexander",NaN,NaN,NaN
22,"Topalov, Veselin",NaN,NaN,NaN
23,"Wei, Yi",NaN,NaN,NaN
25,"Vitiugov, Nikita",NaN,NaN,NaN


In [16]:
# list of players with 'NaN'
lst = list(Wins_filtradas.name)
lst

['Nakamura, Hikaru',
 'Ding, Liren',
 'Firouzja, Alireza',
 'Mamedyarov, Shakhriyar',
 'Le, Quang Liem',
 'Grischuk, Alexander',
 'Topalov, Veselin',
 'Wei, Yi',
 'Vitiugov, Nikita']

In [17]:
nombres_filtrados = []
for n in lst:
    filtro = df.loc[df['name'] == n]
    nombres_filtrados.append(filtro)
    
nombres_filtrados

[               name Wins Draws Losses
 2  Nakamura, Hikaru  NaN   NaN    NaN,
           name           Wins         Draws         Losses
 3  Ding, Liren  760 (33.48 %)  413 (47.4 %)  434 (19.12 %),
                 name            Wins          Draws        Losses
 4  Firouzja, Alireza  1484 (52.68 %)  223 (23.82 %)  662 (23.5 %),
                       name            Wins          Draws          Losses
 15  Mamedyarov, Shakhriyar  2034 (41.27 %)  727 (37.32 %)  1055 (21.41 %),
               name Wins Draws Losses
 16  Le, Quang Liem  NaN   NaN    NaN,
                    name            Wins          Draws        Losses
 17  Grischuk, Alexander  2066 (39.41 %)  805 (41.68 %)  991 (18.9 %),
                 name            Wins          Draws         Losses
 20  Topalov, Veselin  1002 (33.71 %)  None (42.8 %)  698 (23.49 %),
        name           Wins           Draws         Losses
 21  Wei, Yi  704 (39.33 %)  None (41.62 %)  341 (19.05 %),
                 name           Wins    

In [18]:
type(nombres_filtrados)

list

In [19]:
nombres_filtrados[0:2]

[               name Wins Draws Losses
 2  Nakamura, Hikaru  NaN   NaN    NaN,
           name           Wins         Draws         Losses
 3  Ding, Liren  760 (33.48 %)  413 (47.4 %)  434 (19.12 %)]

In [20]:
# convertir la lista de nombres con los valores que faltaban en un dataframe
data_filter = [
    {'name': 'Nakamura, Hikaru', 'Wins': None, 'Draws': None, 'Losses': None},
    {'name': 'Ding, Liren', 'Wins': '760 (33.48 %)', 'Draws': '413 (47.4 %)', 'Losses': '434 (19.12 %)'},
    {'name': 'Firouzja, Alireza', 'Wins': '1484 (52.68 %)', 'Draws': '223 (23.82 %)', 'Losses': '662 (23.5 %)'},
    {'name': 'Mamedyarov, Shakhriyar', 'Wins': '2034 (41.27 %)', 'Draws': '727 (37.32 %)', 'Losses': '1055 (21.41 %)'},
    {'name': 'Le, Quang Liem', 'Wins': None, 'Draws': None, 'Losses': None},
    {'name': 'Grischuk, Alexander', 'Wins': '2066 (39.41 %)', 'Draws': '805 (41.68 %)', 'Losses': '991 (18.9 %)'},
    {'name': 'Topalov, Veselin', 'Wins': '1002 (33.71 %)', 'Draws': 'None (42.8 %)', 'Losses': '698 (23.49 %)'},
    {'name': 'Wei, Yi', 'Wins': '704 (39.33 %)', 'Draws': 'None (41.62 %)', 'Losses': '341 (19.05 %)'},
    {'name': 'Vitiugov, Nikita', 'Wins': '898 (39.27 %)', 'Draws': 'None (45.65 %)', 'Losses': '345 (15.09 %)'}
]

# Crear el DataFrame
filter_name = pd.DataFrame(data_filter)

# Imprimir el DataFrame
print(filter_name)

                     name            Wins           Draws          Losses
0        Nakamura, Hikaru            None            None            None
1             Ding, Liren   760 (33.48 %)    413 (47.4 %)   434 (19.12 %)
2       Firouzja, Alireza  1484 (52.68 %)   223 (23.82 %)    662 (23.5 %)
3  Mamedyarov, Shakhriyar  2034 (41.27 %)   727 (37.32 %)  1055 (21.41 %)
4          Le, Quang Liem            None            None            None
5     Grischuk, Alexander  2066 (39.41 %)   805 (41.68 %)    991 (18.9 %)
6        Topalov, Veselin  1002 (33.71 %)   None (42.8 %)   698 (23.49 %)
7                 Wei, Yi   704 (39.33 %)  None (41.62 %)   341 (19.05 %)
8        Vitiugov, Nikita   898 (39.27 %)  None (45.65 %)   345 (15.09 %)


In [21]:
# Fill the null values with the actual value
data[['Wins', 'Draws', 'Losses']] = data[['Wins', 'Draws', 'Losses']].combine_first(df[['Wins', 'Draws', 'Losses']])
data.head(20)

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Ding, Liren",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
5,"Firouzja, Alireza",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
6,"Nepomniachtchi, Ian",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
7,"So, Wesley",1588 (39.11 %),586 (43.79 %),694 (17.09 %)
8,"Giri, Anish",1226 (32.51 %),688 (48.02 %),734 (19.46 %)
9,Gukesh D,572 (30.94 %),None (22.01 %),870 (47.05 %)


In [22]:
data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    102 non-null    object
 1   Wins    100 non-null    object
 2   Draws   100 non-null    object
 3   Losses  100 non-null    object
dtypes: object(4)
memory usage: 28.1 KB


# inspect the two weird rows with nulls

In [23]:
# Detectar las filas con valores nulos
filas_con_nulos = data[data.isnull().any(axis=1)]
filas_con_nulos

,name,Wins,Draws,Losses
2,"Nakamura, Hikaru",NaN,NaN,NaN
16,"Mamedyarov, Shakhriyar",NaN,NaN,NaN


In [24]:
data.name.value_counts()

Nakamura, Hikaru              2
Mamedyarov, Shakhriyar        2
Carlsen, Magnus               1
Wojtaszek, Radoslaw           1
Adams, Michael                1
                             ..
Sjugirov, Sanan               1
Maghsoodloo, Parham           1
Dubov, Daniil                 1
Harikrishna, Pentala          1
Aravindh, Chithambaram VR.    1
Name: name, Length: 100, dtype: int64

In [25]:
data = data.dropna()
data

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Ding, Liren",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
5,"Firouzja, Alireza",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
...,...,...,...,...
97,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
98,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
99,"Malakhov, Vladimir",919 (41.03 %),None (44.29 %),329 (14.69 %)
100,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [26]:
data = data.reset_index(drop = True)
data

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
3,"Ding, Liren",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
4,"Firouzja, Alireza",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
...,...,...,...,...
95,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
96,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
97,"Malakhov, Vladimir",919 (41.03 %),None (44.29 %),329 (14.69 %)
98,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [27]:
data.columns = ['Name', 'Wins', 'Draws', 'Losses']
data

,Name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
3,"Ding, Liren",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
4,"Firouzja, Alireza",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
...,...,...,...,...
95,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
96,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
97,"Malakhov, Vladimir",919 (41.03 %),None (44.29 %),329 (14.69 %)
98,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [28]:
data.to_csv(r'top_100_wdl.csv', index=False)

# Merge dataframes(final_dataframe)